<font color="magenta" size=7>Source Code

This is the source code that runs the nico's spikesort application.

<font color="red">Arpitha: Like in the mindreader game example, you will create a app that has two `.ipynb` files:
1. A source code file that for all the functions and widgets.
2. An main application file where the interface is neatly displayed.
    
This is the source code file. Finish this before you build the main application. Start with the sort section.

<font color="orange">When running Code, remember to occasionally **delete containers** by running the code in the `Remove Containters` section

**<font color="red">List of Tasks**
<font color="red">  
1. Create a toggle button between mountainsort and spikesort

**Resources**
* [jupyter widgets list](https://ipywidgets.readthedocs.io/en/latest/examples/Widget%20List.html)
* [Spikeforest 2 repository](https://github.com/flatironinstitute/spikeforest2)

# Notebook Setup

## Import Stuff

In [ ]:
import kachery as ka
import os
import subprocess
import sys
import numpy as np
def install(package):
    subprocess.check_call([sys.executable, "-m", "pip", "install", package])
#do things to get braingeneers data
#install("git+https://github.com/braingeneers/braingeneerspy.git")
#install("matplotlib")
import hither_sf as hither
from spikeforest2 import sorters
import matplotlib
import braingeneers.datasets_electrophysiology
import spikeinterface.extractors as se #for mda conversion only
import spiketoolkit as st
import braingeneers
from braingeneers import datasets_electrophysiology as ephys
import numpy as np
import json
import spikeforest2_utils
from spikeforest2_utils import AutoRecordingExtractor, MdaRecordingExtractor
import hither_sf as hither
import kachery as ka
import io
import sys
spikeforest2_utils.__version__=1.0

## Create Directories

In [ ]:
folders = !ls
if "kach_dir" not in folders:
    !mkdir kach_dir
    !touch kach_dir/raw.mda

In [ ]:
current_dir = ! pwd
kach_dir = current_dir[0] + "/kach_dir/"

# <font color="blue">Choose Experiment, time range, channels 

## <font color="blue">choose time range, experiment name, experiment num

 Siddish , please : 
 
     create text box so people can type the batch_uuid into the ui
 
     show users the different experiments in the batch by name and number
         see how to do this under the heading "view other experiment numbers in the same batch". I did it in an archaic sort of way, see if you can make it cleaner/easier to work with. Eventually we may incorporate a function named view_experiments into the braingeneers python package based of implementations we develop here.
 
     create text box to enter experiment number after printing the individual experiments in the batch
     
     if possible, all text boxes and interactables i'm instructing you to make, should show up even if the section is collapsed. We will have to ask elliot about this because he designed wetai and I have no idea how to do this. Regardless, try to position the interactive elements in places where the usability makes sense.

In [ ]:
fname = os.environ['KACHERY_STORAGE_DIR']= kach_dir
batch_uuid = '2020-11-05-e-UCSF-axionplate'
experiment_num= 2 
start = 1 #arpitha, sliders here for start minutes
stop = 5 #and end minutes
#recording_path = 'sha1://961f4a641af64dded4821610189f808f0192de4d/SYNTH_MEAREC_TETRODE/synth_mearec_tetrode_noise10_K10_C4/002_synth.json' #simulated

In [ ]:
#print("this is how long the recording is: ", X.shape[1]/fs/60, " minutes")

 Siddish, please : 

        create a slider bar (one bar) that assigns two integers values [0, max_minutes]. The bar should span [0, max_minutes] for the selected recording, and it will assign start to the first point chosen on the left of the slider bar, and stop to the integer chosen on the right of the slider.

## view other experiment numbers in the same batch

In [ ]:
try:
    for num in range(100):
        print("Below is experiment number ", num)
        experiment = braingeneers.datasets_electrophysiology.load_experiment(batch_uuid, num)
        print(experiment.keys())
        experiment["notes"]
except:
    print("these are the first 100 experiments")

## <font color="blue">choose channels

Siddish please:
    
    create a text box asking users to enter desired channels seperated by commas. So if a user enters "1,9" into the box it will populate the selected_channels array -> [1,9]. If the user does not enter specific channels, make selected_channels a list from [0,max chans). The number of channels, before trimming, is X.shape[0] (32 in this case). So the list should be [0,1,2,{...},31]

In [ ]:
#X.shape[0]

In [ ]:
selected_channels=[5,10]

In [ ]:
len(selected_channels)

## load data, time array (seconds), and sampling freq

In [ ]:
X, t, fs = ephys.load_blocks(batch_uuid, experiment_num)#, start=start, stop=stop) #for the 11-27-20 experiment and raspi expm'ts after november, must specify start stop blocks (minutes)

X = X.T #spikeinterface wants it MxN (channel number by time)
X.shape

In [ ]:
#ephys.create_overview(batch_uuid, experiment_num, with_spikes = False)

In [ ]:
print("this is sampling Freq: ", fs)

In [ ]:
print("this is how long the recording is: ", X.shape[1]/fs/60, " minutes")

## <font color="blue">optionally (ideally): apply geometry for locations of electrodes

In [ ]:
last_chan = X[selected_channels[0],:]
print("last_chan shape ", last_chan.shape)
print("X[0].shape ", X[0].shape)

geom = np.zeros(((len(selected_channels)),2)) # initialize recording device geometry to zeros

if len(selected_channels)==1:
    this_chan = X[selected_channels[0]]
    this_chan = np.array([this_chan])
    X = this_chan
    print("1 chan: ", X.shape)
    
    
if len(selected_channels)==2:
    X = np.append([X[selected_channels[0]]], [X[selected_channels[1]]], axis=0)
    print("2 chan: ", X.shape)
    geom = np.array([[0.0,0.0],[0.0,1.0]])

if len(selected_channels)>2: #PROBLEMATIC?
    for chan in range(1,len(selected_channels)):
        last_chan = np.append([last_chan],[X[selected_channels[chan]]],axis=0)
    X = last_chan


In [ ]:
X.shape

In [ ]:
print(geom)

In [ ]:
#X.shape
#X[1:10,1:10]
print("minutes = ", X.shape[1]/fs/60)
print("expected minutes =", stop-start)

In [ ]:
if X.shape[0] == 2 :
    geom = np.array([[0.0,0.0],[0.0,1.0]])
    print("only two channels, so Geom always two points 1 unit apart.")

In [ ]:
geom.shape #the 2d locations in space of each channel on the recording device

# Create "Recording" for spikeForest and Filter recording

In [ ]:
recording=se.NumpyRecordingExtractor(
    timeseries=X,
    geom=geom,
    sampling_frequency=fs
)

In [ ]:
#recording = st.preprocessing.bandpass_filter(recording, freq_min=300, freq_max=6000)

In [ ]:
fname

raw_path = ka.store_file(fname  + 'raw.mda')

kach_dir_files = ! ls kach_dir
if "geom.csv" not in kach_dir_files :
    ! touch kach_dir/geom.csv

obj = dict(
    raw=raw_path,
    params=ka.load_object(fname + 'params.json'),
    geom=np.genfromtxt(ka.load_file(fname + 'geom.csv'), delimiter=',').tolist()
)

obj['self_reference'] = ka.store_object(obj, basename='{}.json'.format(label))
print(obj['self_reference'])
with open(output_fname, 'w') as f:
    json.dump(obj, f, indent=4)

In [ ]:
def register_recording(*, fname, recording, output_fname, label):
            raw_path = ka.store_file(fname  + 'raw.mda')
            obj = dict(
                raw=raw_path,
                params=ka.load_object(fname + 'params.json'),
                geom=np.genfromtxt(ka.load_file(fname + 'geom.csv'), delimiter=',').tolist()
            )
            obj['self_reference'] = ka.store_object(obj, basename='{}.json'.format(label))

            with open(output_fname, 'w') as f: json.dump(obj, f, indent=4)
            return obj['self_reference']
            
            

In [ ]:
recording_path =  register_recording( recording=recording, fname=fname, output_fname=fname+'new_recording.json', label='new_recording')

# <font color="black"> Sort

In [ ]:
# dummy dataset courtesy of spikeforest's example 
#recording_path = 'sha1://961f4a641af64dded4821610189f808f0192de4d/SYNTH_MEAREC_TETRODE/synth_mearec_tetrode_noise10_K10_C4/002_synth.json' 


<font color="red"> Arpitha, please:
     create 2 buttons, one that says "MountainSort4" and one that says "SpykingCircus". Above the buttons, ask users to "choose your spike sorter."

<font color="red">Arpitha: This is greate work! As an exercise, please reimplement the code above, but instead, use the `toggle button` widget. This is shown below. This does the same thing as what you wrote, but takes less code.

In [ ]:
#Arpitha
#Method 1:
import ipywidgets as ipw #ipywidgets used for button

print ('choose your spike sorter') 

ms4 = ipw.Button(                  #ms4 creates the mountain sort button
    description ='MountainSort4',
    disabled = False,
    button_style='info',           
    tooltip = 'MountainSort4',
    icon = 'check'   
)

def on_ms4_clicked(_):            #define the mountain sort button and globalize so that button does it's function
    global sort_button
    sort_button = 'MountainSort4'
    #print (sort_button)

sc = ipw.Button(
    description ='SpykingCircus',  #sc creates the SpykeCircus sort button
    disabled = False,
    button_style='info', 
    tooltip = 'SpykingCircus',
    icon = 'check' 
)

def on_sc_clicked(_):           #define the SpykeCircus sort button and globalize so that button does it's function
    global sort_button
    sort_button = 'SpykingCircus'
    #print (sort_button)

ms4.on_click(on_ms4_clicked)   #on_click so that button does it's function
sc.on_click(on_sc_clicked)
    
ipw.HBox([ms4,sc])             #HBox so that buttons are on same line
            
            

print(sort_button)

In [ ]:
#Arpitha
#Method 2:
print ('choose your spike sorter') 
test = ipw.ToggleButtons(                                                #used ipw rather than widgets
    options=['MountainSort4', 'SpykingCircus'],
    #description='Choose Algorithm : ',
    button_style='info', # 'success', 'info', 'warning', 'danger' or ''
    tooltips=['Describe mountainsort4', 'describe spikingcircus'],
)
test

In [ ]:
test.value

In [ ]:
#Arpitha
#Method 3
print ('choose your spike sorter') 

button_clicked = 'MountainSort4'                                           #MountainSort 4 is the default button

test = ipw.ToggleButtons(
    options=['MountainSort4', 'SpykingCircus'],
    #description='Choose Algorithm : ',  
    button_style='info', # 'success', 'info', 'warning', 'danger' or ''
    tooltips=['Describe mountainsort4', 'describe spykingcircus'],
)
display(test)                                               

def observe_test (change):
    button_clicked = str(change['new'])
    print(change['new'])                                                   #Set button_clicked to 'new' so that SpykingCircus will come up
    
test.observe (observe_test, names = ['value'])

In [ ]:
print (button_clicked)

In [ ]:
#sort_button = 'MountainSort4' 

if sort_button == 'MountainSort4':
    with ka.config(fr='default_readonly'):
        #with hither.config(cache='default_readwrite'):
            with hither.config(container='default'):
                result = sorters.mountainsort4.run(
                        recording_path=recording_path,
                        sorting_out=hither.File()
                        )
                print("recording out path: ", recording_path)
                print("sorting out path: ",result.outputs.sorting_out)
                sorting_path = str(result.outputs.sorting_out)

    print(result.outputs.sorting_out) 
    

      
if sort_button == 'SpykingCircus':
    with ka.config(fr='default_readonly'):
        #with hither.config(cache='default_readwrite'):
            with hither.config(container='default'):
                result = sorters.spykingcircus.run(
                        recording_path=recording_path,
                        sorting_out=hither.File()
                        )
                print("recording out path: ", recording_path)
                print("sorting out path: ",result.outputs.sorting_out)
                sorting_path = str(result.outputs.sorting_out)

print(result.outputs.sorting_out)


In [ ]:
sorting_out_short = sorting_path[12:-1]

print("This is the new short path for the sorting: ",sorting_out_short)

In [ ]:
from spikeforest2_utils import AutoRecordingExtractor, AutoSortingExtractor
import kachery as ka

universal_sorting_path = ka.store_file(sorting_out_short)

recording = AutoRecordingExtractor(recording_path, download=False)
sorting_true = AutoSortingExtractor(universal_sorting_path)

In [ ]:
# Now you can access the recording using the SpikeInterface API
# For example, print the electrode locations and unit IDs
print(recording.get_channel_locations())
print(sorting_true.get_unit_ids())

In [ ]:
import pickle

In [ ]:
import spikeinterface.widgets as sw

# Make Plots

## <font color="brown"> Scratch Paper

Code for clearing output [comes from here](https://stackoverflow.com/questions/42782902/clear-widget-area-of-a-cell-in-a-jupyter-notebook-from-within-notebook)

In [ ]:
from IPython.display import display, clear_output

Create Toggle button example

In [ ]:
test= ipw.ToggleButtons(
    options=['1', '2'],
    #description='Choose Algorithm : ',
    #button_style="", # 'success', 'info', 'warning', 'danger' or ''
    #tooltips=['Describe mountainsort4', 'describe spikingcircus'],
)
test

In [ ]:
#w_rs = sw.plot_rasters(sorting_true, sampling_frequency=fs)
    
#w_isi = sw.plot_isi_distribution(sorting_true, sampling_frequency=fs, bins=10, window=1)

Build observe function that change a display based on which button is clicked

In [ ]:
def observe_test(change):
    # Clear the previous plot output and then redisplay buttons
    clear_output()
    display( test )
    
    # Run the main function
    if test.value == '1':
        print("This is an explanation about 1")
        sw.plot_rasters(sorting_true, sampling_frequency=fs)
    else:
        print("Here's a different explanation about 2")
        sw.plot_isi_distribution(sorting_true, sampling_frequency=fs, bins=10, window=1)


test.observe( observe_test , names = 'value')   

**Expaiing what display does**

<font color="red">wrong way

In [ ]:
ipw.IntSlider() 
ipw.Text() 

correct way

In [ ]:
display( ipw.IntSlider() )
display( ipw.Text() )

## Main Section

 Elliot, 
Can we get these summary figures to render even if the dropdown for Make Plots is collapsed?

<font color="red">Arpitha: Create `toggle buttons` or possibly regular buttons that when changed, switch between the different graphs. To do this with the toggle buttons I think you need to use observe function.

example with toggle button

In [ ]:
test= ipw.ToggleButtons(
    options=['Time Series', 'Spectrum', "isi distribution", "rasters"],
    #description='Choose Algorithm : ',
    #button_style="", # 'success', 'info', 'warning', 'danger' or ''
    #tooltips=['Describe mountainsort4', 'describe spikingcircus'],
)
test

example with regular button

In [ ]:
time_series = ipw.Button(
    description ='time series',
    disabled = False,
    button_style='info', 
    tooltip = 'time serires',
    #icon = 'check'   
)

def on_time_series(_):
    sw.plot_rasters(sorting_true, sampling_frequency=fs)
time_series.on_click(on_time_series)

time_series


Arpitha Please,

    For these plots, can we use pyplot (or maybe another package) to 
        for the plot_amplitudes_timeseries figure:
            -add time in s on the X-axis. 
        for the plot_unit_waveforms figure:
            -add time in ms to graph. The length of the X-axis here should be 3ms; if that's not clear from the axis values let me know.
            -Add the y axis units in microvolts. FYI, you can see in the plot_amplitudes_timeseries graph that unit 1 has a lot of spikes around 10 microvolts.
            
    Note: it is ok for now if there is extra text printing, or other imperfections in the code. Lets focus on the implementation then we can perfect it. Let me know when you are done. Thanks!
    
    P.S. Theres an example under the curate_units graph. Notice how I use plt.yticks to change the scaling of the graph. Try to interact with the graphs after their creation to refurbish them. More information about what the figure objects may be useful here: https://github.com/SpikeInterface/spikewidgets/tree/master/spikewidgets/widgets . For example the unitwaveforms.py has the plot_unit_waveforms function definition which shows how they created the figure. Inspecting them can give you hints how to understand/manipulate the figures.
    
   

In [ ]:
w_ts = sw.plot_timeseries(recording)
    
    #does not work w no geom:
    #w_el = sw.plot_electrode_geometry(recording)
    #pickle.dump(w_el, output, pickle.HIGHEST_PROTOCOL)
    
w_sp = sw.plot_spectrum(recording)
    
w_rs = sw.plot_rasters(sorting_true, sampling_frequency=fs)
    
w_isi = sw.plot_isi_distribution(sorting_true, sampling_frequency=fs, bins=10, window=1)
    
w_ach = sw.plot_autocorrelograms(sorting_true, sampling_frequency=fs, bin_size=1, window=10, unit_ids=[1, 2, 4, 5, 8, 10, 7])
    
w_cch = sw.plot_crosscorrelograms(sorting_true, sampling_frequency=fs, unit_ids=[1, 5, 8], bin_size=0.1, window=5)

w_wf = sw.plot_unit_waveforms(recording, sorting_true,unit_ids=sorting_true.get_unit_ids(),  max_spikes_per_unit=100)
    
w_ampd = sw.plot_amplitudes_distribution(recording, sorting_true, max_spikes_per_unit=300)
    
w_ampt = sw.plot_amplitudes_timeseries(recording, sorting_true, max_spikes_per_unit=300)
    

In [ ]:
ipw.ToggleButtons(
    options = ['Plot_amplitudes_timeseries','Plot_unit_waveforms'],
    #description = 'Choose Algorithm:',
    button_style ='info',
    tooltips = ['Describe Plot_amplitudes_timeseries ','Describe Plot_unit_waveforms '],
)
#Use the observe function

# Curate Units

Arpitha, please:
    
    create a text box asking users to enter curated units seperated by commas. So if a user enters "1,9" into the box it will populate the curated_units array -> [1,9]. If the user does not enter specific channels, make curated_units equal to sorting_true.get_unit_ids()

<font color="red">Arptiha: I think it's better to create 2 integer boxes.

In [ ]:
display( widgets.IntText( description="start", value=1 ) )
display( widgets.IntText( description="stop", value=1 ) )

<font color="red">Perhaps later you can try making it fancier with an `intRangeSlider`

In [ ]:

widgets.IntRangeSlider(
    value=[3,7],
    min=0,
    max=10,
    step=1,
    description='Test:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d',
)

In [ ]:
# Arpitha
#import array as arr

#curated_units = arr.array('i',[1,1])

curated_units = [0,0]                    #Used list instead of array
curated_units[0] = widgets.IntText(
        step = 1,                        #Did not make text box bounded
        description = 'Start:',
        disabled = False
)
curated_units[1] = widgets.IntText(
    step = 1,
    description = 'Stop:',
    disabled = False
)
widgets.HBox([curated_units[0], curated_units[1]])


In [ ]:
# Arpitha

print (curated_units[0].value)                                       
print (curated_units[1].value)

if (curated_units[0].value==0 and curated_units[1].value==0):
    curated_units = sorting_true.get_unit_ids()                  #If nothing entered in textbox, curated_units equals sorting ids



In [ ]:
# Arpitha

curated_units = widgets.IntRangeSlider(
    value=[3,7],
    min=1,
    max=9,
    step=1,
    description='Range',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d',
)
display (curated_units)
#make curated_units equal to sorting_true.get_unit_ids()

In [ ]:
# Arpitha

print (curated_units.value[0])
print (curated_units.value[1])

if (curated_units.value[0]==0 and curated_units.value[1]==0):
    curated_units = sorting_true.get_unit_ids()


In [ ]:
import matplotlib.pyplot as plt

w_wf = sw.plot_unit_waveforms(recording, sorting_true,unit_ids=curated_units,max_channels=1,color='darkgreen', max_spikes_per_unit=300)#,figure=fig)#,axis=)

plt.yticks(np.arange(-1.5, 2.0, 1))
print("Each graph spans 3ms x axis. Y axis is microvolts -- the same as the amplitude distributions in the other graphs")
w_wf = sw.plot_unit_waveforms(recording, sorting_true,unit_ids=[curated_units[1],curated_units[0]],max_channels=1,color='darkgreen', max_spikes_per_unit=300)#,figure=fig)#,axis=)

plt.yticks(np.arange(-1.5, 2.0, 1))

# Remove Containers

In [ ]:
!sudo docker ps -a | grep Exit | cut -d ' ' -f 1 | xargs sudo docker rm